# GGH数字签名
**密钥生成算法：**  
输入：维度n  
输出：私钥sk，公钥pk
1. 生成1个好基$V$, $V \leftarrow \mathbb{Z}^{n \times n}, \mathcal{H}(V) \approx 1$
2. 生成1个坏基$W$，$\mathcal{H}(W) \approx 1$
3. 私钥sk$\leftarrow V$
4. 公钥pk$\leftarrow W$
5. 返回(sk, pk)
----
**数字签名：**  
输入：私钥sk，明文m  
输出：签名s  
1. 用Babai算法求解s $\leftarrow$ solveCVP(V, m)
2. 返回s
----
**验证签名：**  
输入：签名s，明文m  
输出：接受或拒绝  
1. 如果$||s-m|| < \sqrt{n} \sigma(\mathcal{L})$ 则接受
2. 否则，拒绝

**例：** 用三维格对GGH数字签名进行举例。  
考虑一个三维的格$\mathcal{L} \subset \mathbb{R}^3$，用一组“好”基为$$V = 
\left [ \begin{array}{ccc} 
100 & 0 & 0 \\
0 & 90 & 15 \\
0 & 20 & 95
\end{array} \right ]$$
生成“坏”基的幺模矩阵为$$U = 
\left [ \begin{array}{ccc} 
50 & 11 & 2 \\
-5 & -20 & -7 \\
2 & 3 & 1
\end{array} \right ]$$
签名明文为$$m=\left [ \begin{array}{ccc} 
368 & 465 & 593 
\end{array} \right ]$$

#### 生成密钥 

In [1]:
import numpy as np
import simchain.lbc as lbc
# 私钥
good = np.array([[100, 0, 0], [0, 90, 15], [0, 20, 95]])
good

array([[100,   0,   0],
       [  0,  90,  15],
       [  0,  20,  95]])

2019-04-30 17:36:46,363 - Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
# 幺模矩阵
u = np.array([[50, 11, 2], [-5, -20, -7], [2, 3, 1]])
u

array([[ 50,  11,   2],
       [ -5, -20,  -7],
       [  2,   3,   1]])

In [3]:
# 公钥
bad = np.dot(u, good)
bad

array([[ 5000,  1030,   355],
       [ -500, -1940,  -965],
       [  200,   290,   140]])

In [4]:
# 公钥的Hadamard比
lbc.hadamard(good)

0.9762428656955312

In [5]:
# 私钥的Hadamard比
lbc.hadamard(bad)

0.04147703489649823

In [6]:
# 幺模矩阵行列式
np.linalg.det(u)

1.000000000000136

#### 数字签名

In [7]:
# 明文向量
m = [368, 465, 593]
x = np.linalg.solve(good, m)
x = np.around(x).astype('int')
x

array([4, 4, 5])

In [8]:
# 签名向量
s = np.dot(good, x)
s

array([400, 435, 555])

#### 验证签名

In [9]:
# 签名与明文的距离
s_m = np.linalg.norm(s-m)
s_m

58.034472514187634

In [10]:
# 公钥的行列式绝对值
bad_det = abs(np.linalg.det(bad))
sigma_L = np.sqrt(3/(2 * np.pi * np.e)) * bad_det ** (1./3)
sigma_L

39.30744074553697

In [11]:
n_sigma_L = np.sqrt(3) * sigma_L
n_sigma_L

68.0824844867731

In [12]:
# 验证签名成功
s_m <= n_sigma_L

True

----
# Lyubashevshy数字签名

**密钥生成算法：**  
输入：整数d, n, m, k, 模数q  
输出：私钥sk，公钥pk  
1. 私钥sk $\leftarrow(S \leftarrow \{-d,\dots,0,\dots,d\}^{m \times k})$
2. 公钥pk $\leftarrow(A \leftarrow \mathbb{Z}_q^{n \times m},T=AS mod q)$
3. 返回(sk, pk)
----
**数字签名：**  
输入：签名明文me，常数M，私钥S，公钥A  
输出：签名(z, c)  
1. 随机生成向量$y \leftarrow D_\sigma^m$
2. 计算哈希值$c \leftarrow H(Ay mod q, me)$
3. 计算签名$z \leftarrow Sc+y$
4. 生成随机数$r \leftarrow [0,1]$
5. 如果$r < \frac{D_\sigma^m(z)}{MD_{SC,\sigma}^m(z)}$
6. 否则，重复1~5
----
**验证签名：**  
输入：签名(z,c)，明文me  
输出：接受或拒绝  
1. 如果$||Z||<2\sigma \sqrt{m}且c=H(Az-Tz mod q, me)$，接受
2. 否则，拒绝

#### 生成密钥

In [13]:
import numpy as np
from simchain.lbc import convert_to_Zq

# 定义常数
d, n, m, k, q, sigma = 30, 5, 6, 7, 21, 10
# 随机生成私钥矩阵S
S = np.random.randint(-d, d+1, (m, k))
S

array([[  2, -24,  30,  26, -14, -23, -15],
       [ 18,   8,  12,  14,  13,  -8,  -2],
       [-22, -17,  21,  -2,  -5,  -9,  25],
       [ 21,  -8,  19,  -9,  -2,  11,  17],
       [-25,  26,  -6, -14, -11,   7,   8],
       [-30, -25, -27,   4,  12,  -1,  29]])

In [14]:
# 转换S中的元素
S = convert_to_Zq(S, q)
S

array([[  2,  -3,   9,   5,   7,  -2,   6],
       [ -3,   8,  -9,  -7,  -8,  -8,  -2],
       [ -1,   4,   0,  -2,  -5,  -9,   4],
       [  0,  -8,  -2,  -9,  -2, -10,  -4],
       [ -4,   5,  -6,   7,  10,   7,   8],
       [ -9,  -4,  -6,   4,  -9,  -1,   8]])

In [15]:
# 生成公钥矩阵A
A = np.random.randint(-d, d+1, (n, m))
A

array([[ -1, -26,  26, -16, -14,  28],
       [ 19,  -7, -25, -18,  20,  21],
       [  6,  21, -10, -27,   5,  12],
       [ 24,  14, -20,  20,  10, -15],
       [ -9, -20,  16,  25,  -4,  16]])

In [16]:
# 转换A中的元素
A = convert_to_Zq(A, q)
A

array([[ -1,  -5,   5,   5,   7,   7],
       [ -2,  -7,  -4,   3,  -1,   0],
       [  6,   0, -10,  -6,   5,  -9],
       [  3,  -7,   1,  -1,  10,   6],
       [ -9,   1,  -5,   4,  -4,  -5]])

In [17]:
# 生成公钥矩阵T
T = np.dot(A, S)
T

array([[ -83,  -50,  -58,   52,    5,  -11,  116],
       [  25,  -95,   45,   13,   46,   59,  -34],
       [  83,   51,   90,  103,  235,  182,  -12],
       [ -68,  -27,   -4,  165,  120,  115,  168],
       [  45,  -17,  -44, -126,  -49,   -8, -164]])

In [18]:
# 转换T中的元素
T = convert_to_Zq(T, q)
T

array([[  1,  -8,   5,  10,   5,  10, -10],
       [  4,  10,   3,  -8,   4,  -4,   8],
       [ -1,   9,   6,  -2,   4,  -7,   9],
       [ -5,  -6,  -4,  -3,  -6,  10,   0],
       [  3,   4,  -2,   0,  -7,  -8,   4]])

#### 数字签名

In [19]:
from math import exp, sqrt
from simchain.lbc import hash_to_baseb

# 选择常数M
M = 1
# 选择签名明文
message = b'111'
# 开始签名
while True:
    # 创建随机向量y
    y = np.random.normal(0, sigma, m).astype('int')
    # 计算Ay
    Ay = np.dot(A, y)
    # 计算Ay mod p
    Ay = convert_to_Zq(Ay, q)
    # 计算Ay和签名明文的哈希值
    c = hash_to_baseb(Ay, message, k, 3)
    # 计算Sc
    Sc = np.dot(S, c)
    # 计算向量z
    z = Sc + y
    pxe = -z.dot(z) + y.dot(y)
    val = exp(pxe / (2*sigma**2)) / M
    if np.random.rand() < min(val , 1):
        break
z

array([  0, -21,  18, -12,   1, -12])

In [20]:
c

array([2, 2, 2, 1, 0, 0, 1])

#### 验证签名

In [21]:
np.linalg.norm(z) <= 2 * sigma * sqrt(m)

True

In [22]:
# 计算Az - Tc
AzTc = np.dot(A, z) - np.dot(T, c)
# 计算(Az - Tc) mod q
AzTc = convert_to_Zq(AzTc, q)
# 计算Az-Tc与明文的哈希值
hc = hash_to_baseb(AzTc, message, k, 3)
# 验证哈希值是否相等
np.allclose(c, hc)

True

哈希函数hash_to_baseb()的实现

In [23]:
from numpy import array_str, array
from hashlib import sha512

# 输入为矩阵，明文，输出向量的长度以及向量元素的范围[0, b)
def hash_to_baseb(matrix, message,k,b=3):
    
    # 组合矩阵字节串和明文，并对其进行sha512运算，返回十六进制哈希值
    hexval = sha512(array_str(matrix).encode() + message).hexdigest()
    # 将十六进制字符串编码成向量，向量的元素范围为[0, b)，返回向量的前k个元素
    return array(list(map(int, list(b2b(hexval, 16, b)[:k]))))

base_symbols='0123456789abcdefghijklmnopqrstuvwxyz'

# 将整数编码到字符为[0, b)上的字符串
def v2r(n, b): 
    digits = ''
    while n > 0:
        digits = base_symbols[n % b] + digits
        n  = n // b
    return digits

# 将十六进制字符串编码到字符为[0, b)上的字符串
def b2b(digits, b1, b2):
    return v2r(r2v(digits, b1), b2)

In [24]:
import numpy as np
from simchain.lbc.utils import b2b
import hashlib

# 创建随机矩阵
matrix = np.random.randint(-5, 5, (3, 4))
# 选择明文
message = b'I love blockchain'
# 组合矩阵和明文
string = np.array_str(matrix).encode() + message
string

b'[[ 3 -5  1 -1]\n [-5 -1  0  0]\n [-5 -3  3 -4]]I love blockchain'

In [25]:
# 对矩阵进行sha512哈希运算
hexval = hashlib.sha512(string).hexdigest()
hexval

'5c72a8cafaca0b61ec8c067a9525b4ad2a2a443d36050928611e82ef6e6a821c0a2cb96626bf9b08b7635b0ff27af345bb0b674949a294871a53113078fafd79'

In [26]:
# 将返回的字符串编码到字符为[0, 3)的字符串
s = b2b(hexval, 16, 3)
s

'10101022000101221220120112201201002210012000220221022202211200200021121121200112022000200202022001020110011022021012000202011022202010112121221202211202220101020020100112200121121002012102002021121120202201112121101210201212022021100122220020200012121122221101202212000120011020200000222010102111222101012201212022221210002'

In [27]:
# 将字符串s转换为整数列表
val = map(int, list(s))

In [28]:
# 取列表的前k=10个元素，并转换为ndarra
vec = np.array(list(val)[:10])
vec

array([1, 0, 1, 0, 1, 0, 2, 2, 0, 0])